In [1]:
import pandas as pd
raw_data = pd.read_csv('data.csv',sep = ",")

raw_data.dropna(how="any", axis = 0, inplace = True)
raw_data = raw_data.reset_index()

#remove all columns that contain data about each fighters previous opponents
opponents_col = []

for column in raw_data: 
    if 'opp' in column:
        opponents_col.append(column)

raw_data.drop(opponents_col, axis=1, inplace = True)

#no need to track the subtype of decision victories
red_list = ['R_win_by_Decision_Majority', 'R_win_by_Decision_Split','R_win_by_Decision_Unanimous']
blue_list = ['B_win_by_Decision_Majority', 'B_win_by_Decision_Split','B_win_by_Decision_Unanimous']

raw_data['R_win_by_Decision'] = raw_data[red_list].sum(axis=1)
raw_data['B_win_by_Decision'] = raw_data[blue_list].sum(axis=1)

raw_data.drop(red_list, axis=1, inplace = True)
raw_data.drop(blue_list, axis=1, inplace = True)

#remove the column 'number of rounds', we are predicting based on past data, not current data
raw_data.drop(['no_of_rounds'], axis=1, inplace = True)

#total rounds fought and total time fought both essentially measure the same feature
raw_data.drop(['B_total_time_fought(seconds)', 'R_total_time_fought(seconds)'], axis=1, inplace = True)

#idk wether to remove these or not
#PASS is no. times the guard was passed, REV is the no. of Reversals landed (seem like too specific stats)
to_remove = ['R_avg_PASS', 'B_avg_PASS', 'R_avg_REV', 'B_avg_REV']

#significant strikes landed and attempted are further broken down into subcategories by
#body part : BODY, LEG, HEAD and distance: CLINCH, GROUND, DISTANCE
#I think that this is unnecessary

to_remove.extend([
    'B_avg_BODY_att', 'B_avg_BODY_landed','R_avg_BODY_att', 'R_avg_BODY_landed',
    'B_avg_LEG_att', 'B_avg_LEG_landed', 'R_avg_LEG_att', 'R_avg_LEG_landed',
    'B_avg_HEAD_att', 'B_avg_HEAD_landed', 'R_avg_HEAD_att', 'R_avg_HEAD_landed'
    ])

to_remove.extend([
    'B_avg_CLINCH_att', 'B_avg_CLINCH_landed','R_avg_CLINCH_att', 'R_avg_CLINCH_landed',
    'B_avg_GROUND_att', 'B_avg_GROUND_landed', 'R_avg_GROUND_att', 'R_avg_GROUND_landed',
    'B_avg_DISTANCE_att', 'B_avg_DISTANCE_landed', 'R_avg_DISTANCE_att', 'R_avg_DISTANCE_landed'
    ])

raw_data.drop(to_remove, axis=1, inplace = True)

#create new column for average total striking accuracy
blue_total_att = raw_data['B_avg_TOTAL_STR_att']
blue_total_landed = raw_data['B_avg_TOTAL_STR_landed']
red_total_att = raw_data['R_avg_TOTAL_STR_att']
red_total_landed = raw_data['R_avg_TOTAL_STR_landed']
blue_total_pct = []
red_total_pct = []

for i in range(len(blue_total_att)):
    att = blue_total_att[i]
    landed = blue_total_landed[i]
    if (att == 0):
        blue_total_pct.append(0)
    else:
        blue_total_pct.append(landed / att)

for i in range(len(red_total_att)):
    att = red_total_att[i]
    landed = red_total_landed[i]
    if (att == 0):
        red_total_pct.append(0)
    else:
        red_total_pct.append(landed / att)

raw_data['B_avg_TOTAL_STR_pct'] = blue_total_pct
raw_data['R_avg_TOTAL_STR_pct'] = red_total_pct

# Cleaning the Winner column (red fighter = 1; blue fighter = 0)
new_winners = []

raw_data = raw_data[raw_data['Winner'] != 'Draw']

raw_data = raw_data.reset_index()

winners = raw_data['Winner']

for i in winners:
    if i == 'Red':
        new_winners.append(1)
    else:
        new_winners.append(0)

raw_data['Winner'] = new_winners

#add column for gender
gender = []
weight_class = raw_data['weight_class']

for i in weight_class:
    if 'Women' in i:
        gender.append('F')
    else:
        gender.append('M')

raw_data['gender'] = gender

#add column for win percentage 
blue_wins = raw_data['B_wins']
blue_losses = raw_data['B_losses']
blue_draws = raw_data['B_draw']
red_wins = raw_data['R_wins']
red_losses = raw_data['R_losses']
red_draws = raw_data['R_draw']
blue_win_pct = []
red_win_pct = []

for i in range(len(blue_wins)):
    wins = blue_wins[i]
    losses = blue_losses[i]
    draws = blue_draws[i]
    total = wins + losses + draws
    if (total == 0):
        blue_win_pct.append(0)
    else:
        blue_win_pct.append(wins / total)

for i in range(len(red_wins)):
    wins = red_wins[i]
    losses = red_losses[i]
    draws = red_draws[i]
    total = wins + losses + draws
    if (total == 0):
        red_win_pct.append(0)
    else:
        red_win_pct.append(wins / total)

raw_data['B_win_pct'] = blue_win_pct
raw_data['R_win_pct'] = red_win_pct

#add column for height, weight, reach differential 
blue_height = raw_data['B_Height_cms']
blue_weight = raw_data['B_Weight_lbs']
blue_reach = raw_data['B_Reach_cms']
red_height = raw_data['R_Height_cms']
red_weight = raw_data['R_Weight_lbs']
red_reach = raw_data['R_Reach_cms']
height_diff = []
weight_diff = []
reach_diff = []

for i in range(len(blue_height)):
    height_diff.append(red_height[i] - blue_height[i])
    weight_diff.append(red_weight[i] - blue_weight[i])
    reach_diff.append(red_reach[i] - blue_reach[i])

raw_data['height_diff'] = height_diff
raw_data['weight_diff'] = weight_diff
raw_data['reach_diff'] = reach_diff

#check
#print(raw_data)
#for column in raw_data:
#    print(column)


In [2]:
#perform logistic regression
#y variable: 1 (red corner winner of fight), 0: (blue corner winner of fight)

#first drop non-numeric columns
#Winner column (predicted variable) is alredy 0,1 encoded

to_remove = [
    'R_fighter', 'B_fighter', 'Referee', 'date', 'location', 'title_bout', 'weight_class',
    'R_Stance', 'B_Stance', 'level_0', 'index', 'gender'
]

raw_data.drop(to_remove, axis=1, inplace = True)

print(list(raw_data.columns))


['Winner', 'B_current_lose_streak', 'B_current_win_streak', 'B_draw', 'B_avg_KD', 'B_avg_SIG_STR_att', 'B_avg_SIG_STR_landed', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT', 'B_avg_TD_att', 'B_avg_TD_landed', 'B_avg_TD_pct', 'B_avg_TOTAL_STR_att', 'B_avg_TOTAL_STR_landed', 'B_longest_win_streak', 'B_losses', 'B_total_rounds_fought', 'B_total_title_bouts', 'B_win_by_KO/TKO', 'B_win_by_Submission', 'B_wins', 'B_Height_cms', 'B_Reach_cms', 'B_Weight_lbs', 'R_current_lose_streak', 'R_current_win_streak', 'R_draw', 'R_avg_KD', 'R_avg_SIG_STR_att', 'R_avg_SIG_STR_landed', 'R_avg_SIG_STR_pct', 'R_avg_SUB_ATT', 'R_avg_TD_att', 'R_avg_TD_landed', 'R_avg_TD_pct', 'R_avg_TOTAL_STR_att', 'R_avg_TOTAL_STR_landed', 'R_longest_win_streak', 'R_losses', 'R_total_rounds_fought', 'R_total_title_bouts', 'R_win_by_KO/TKO', 'R_win_by_Submission', 'R_wins', 'R_Height_cms', 'R_Reach_cms', 'R_Weight_lbs', 'B_age', 'R_age', 'R_win_by_Decision', 'B_win_by_Decision', 'B_avg_TOTAL_STR_pct', 'R_avg_TOTAL_STR_pct', 'B_win_pc

In [3]:
#feature selection
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression
from sklearn import metrics
from sklearn.model_selection import train_test_split

#retrieve dependent(y) and independent (X) column(s) from the dataframe
y = raw_data['Winner']
X = raw_data.drop('Winner',axis=1)

#select 20 most important features
logreg = LogisticRegression(solver = 'newton-cg')
rfe = RFE(logreg, 20)
rfe = rfe.fit(X, y)

mask = rfe.support_
columns = list(raw_data.columns)
columns.remove('Winner')
selected_columns = []

for i in range(len(mask)):
    if mask[i]:
        selected_columns.append(columns[i])

print(selected_columns)

['B_current_win_streak', 'B_avg_SIG_STR_pct', 'B_avg_SUB_ATT', 'B_avg_TD_att', 'B_total_title_bouts', 'B_win_by_KO/TKO', 'B_win_by_Submission', 'B_wins', 'R_avg_SIG_STR_att', 'R_avg_SIG_STR_landed', 'R_avg_TD_landed', 'R_avg_TD_pct', 'R_avg_TOTAL_STR_att', 'R_avg_TOTAL_STR_landed', 'R_losses', 'R_total_title_bouts', 'R_age', 'B_win_by_Decision', 'B_avg_TOTAL_STR_pct', 'R_avg_TOTAL_STR_pct']


In [4]:
#statistical analysis
import statsmodels.api as sm

#select only the top 20 features (columns)
X = raw_data[selected_columns]

logit_model=sm.Logit(y,X)
result = logit_model.fit()
print(result.summary())

Optimization terminated successfully.
         Current function value: 0.615980
         Iterations 6
                           Logit Regression Results                           
Dep. Variable:                 Winner   No. Observations:                 3151
Model:                          Logit   Df Residuals:                     3131
Method:                           MLE   Df Model:                           19
Date:                Mon, 16 Dec 2019   Pseudo R-squ.:                 0.05746
Time:                        13:47:37   Log-Likelihood:                -1941.0
converged:                       True   LL-Null:                       -2059.3
Covariance Type:            nonrobust   LLR p-value:                 1.527e-39
                             coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------------------
B_current_win_streak      -0.0872      0.030     -2.893      0.004      -0.146      

In [5]:
#from the above statistical analysis, 2 features (B_avg_SUB_ATT, R_avg_TD_pct) have p-values > 0.05, 
#so we will remove those

selected_columns.remove('B_avg_SUB_ATT')
selected_columns.remove('R_avg_TD_pct')
selected_columns.remove('B_win_by_KO/TKO')

print(list(selected_columns))


X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

logreg = LogisticRegression()
logreg.fit(X_train, y_train)

y_pred = logreg.predict(X_test)

print(metrics.accuracy_score(y_test, y_pred))

['B_current_win_streak', 'B_avg_SIG_STR_pct', 'B_avg_TD_att', 'B_total_title_bouts', 'B_win_by_Submission', 'B_wins', 'R_avg_SIG_STR_att', 'R_avg_SIG_STR_landed', 'R_avg_TD_landed', 'R_avg_TOTAL_STR_att', 'R_avg_TOTAL_STR_landed', 'R_losses', 'R_total_title_bouts', 'R_age', 'B_win_by_Decision', 'B_avg_TOTAL_STR_pct', 'R_avg_TOTAL_STR_pct']
0.6501057082452432


/Users/Zuhao_17/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [6]:
cnf_matrix = metrics.confusion_matrix(y_test, y_pred)
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns

class_names=[0,1] # name  of classes
fig, ax = plt.subplots(figsize=(5,5))  
tick_marks = np.arange(len(class_names))
plt.xticks(tick_marks, class_names)
plt.yticks(tick_marks, class_names)
# create heatmap

sns.heatmap(pd.DataFrame(cnf_matrix), annot=True, cmap="YlGnBu" ,fmt='g')
ax.xaxis.set_label_position("top")
plt.tight_layout()
plt.title('Confusion matrix', y=1.1)
plt.ylabel('Actual label')
plt.xlabel('Predicted label')

print("Accuracy:",metrics.accuracy_score(y_test, y_pred))
print("Precision:",metrics.precision_score(y_test, y_pred))

Accuracy: 0.6501057082452432
Precision: 0.671990171990172


In [7]:
from sklearn.metrics import accuracy_score
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import cross_val_score


knn = KNeighborsClassifier(n_neighbors=19)
knn_scores = cross_val_score(knn, X, y, cv=10, scoring='accuracy')

lda = LinearDiscriminantAnalysis()
lda_scores = cross_val_score(lda, X, y, cv=10, scoring='accuracy')

print("K-nearest neighbours classifier accuracy: %0.2f (+/- %0.2f)" % (knn_scores.mean(), knn_scores.std() * 2))
print("Linear Discriminant Analysis (LDA) classifier accuracy: %0.2f (+/- %0.2f)" % 
      (lda_scores.mean(), lda_scores.std() * 2))




K-nearest neighbours classifier accuracy: 0.61 (+/- 0.05)
Linear Discriminant Analysis (LDA) classifier accuracy: 0.65 (+/- 0.07)
